In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import common.common_machine_learning as common
import common.feature_num as feature_num
import common.features_datasets_externos as feature

In [2]:
train = pd.read_csv('sets_de_datos/train.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/test.csv', index_col = 0)

In [3]:
train.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

### Promedio de precio para cantidad de baños según ciudad

In [4]:
banos_preciopromedio = train.groupby(["ciudad", "banos"])["precio"].mean().to_frame()

In [5]:
banos_preciopromedio = banos_preciopromedio.reset_index()

In [6]:
#banos_cantidad.reset_index().set_index("ciudad")
banos_preciopromedio = banos_preciopromedio.pivot_table(values='precio', index=banos_preciopromedio["ciudad"], columns='banos', aggfunc='first')

In [7]:
banos_preciopromedio.head()

banos,1.0,2.0,3.0,4.0
ciudad,,,,
Abasolo,1.907800e+06,1.156833e+06,NaN,8.500000e+06
Abejones,2.500000e+06,NaN,NaN,NaN
Acajete,2.368686e+06,NaN,3.127500e+06,NaN
Acambay,2.083333e+06,1.270000e+06,NaN,NaN
Acapulco de Juárez,8.228125e+05,1.823339e+06,3.573077e+06,5.311640e+06


### Promedio de precio para cantidad de habitaciones según ciudad

In [8]:
habitaciones_preciopromedio = train.groupby(["ciudad", "habitaciones"])["precio"].mean().to_frame()

In [9]:
habitaciones_preciopromedio = habitaciones_preciopromedio.reset_index()

In [10]:
habitaciones_preciopromedio = habitaciones_preciopromedio.pivot_table(values='precio', index=habitaciones_preciopromedio["ciudad"], columns='habitaciones', aggfunc="first")

In [11]:
habitaciones_preciopromedio.head()

habitaciones,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
ciudad,,,,,,,,,,
Abasolo,NaN,1.252800e+06,1.590000e+06,1.600500e+06,NaN,NaN,NaN,NaN,NaN,8500000.0
Abejones,NaN,NaN,2.500000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acajete,2.368686e+06,5.500000e+06,2.336667e+06,NaN,5.800000e+06,NaN,NaN,NaN,NaN,NaN
Acambay,NaN,6.250000e+05,2.030000e+06,1.200000e+06,NaN,NaN,NaN,NaN,NaN,NaN
Acapulco de Juárez,1.185437e+06,1.374582e+06,2.791943e+06,5.201431e+06,3.336639e+06,2.865238e+06,2025000.0,4225000.0,NaN,5750000.0


### Promedio de precio para cantidad de garages según ciudad

In [12]:
garages_preciopromedio = train.groupby(["ciudad", "garages"])["precio"].mean().to_frame()

In [13]:
garages_preciopromedio = garages_preciopromedio.reset_index()

In [14]:
garages_preciopromedio = garages_preciopromedio.pivot_table(values='precio', index=garages_preciopromedio["ciudad"], columns='garages', aggfunc="first")

In [15]:
garages_preciopromedio.head()

garages,0.0,1.0,2.0,3.0
ciudad,,,,
Abalá,3.150000e+05,NaN,NaN,NaN
Abasolo,1.140250e+06,NaN,4.063333e+06,NaN
Acajete,NaN,2.368686e+06,3.662000e+06,NaN
Acambay,NaN,4.500000e+05,1.240000e+06,9.500000e+05
Acapulco de Juárez,2.290756e+06,1.748686e+06,3.692571e+06,4.235587e+06


In [16]:
def asignar_caracteristica_precio_promedio(df, df_caracteristica, caracteristica, nombre):
    precio_promedio = []
    for index, row in train.iterrows():
        try:
            ciudad = row["ciudad"]
            cantidad_caracteristica = row[caracteristica]
            precio = df_caracteristica.loc[ciudad][cantidad_caracteristica]
        except:
            precio = np.nan
        precio_promedio.append(precio)
    df[nombre] = precio_promedio
    return df

In [17]:
train = asignar_caracteristica_precio_promedio(train, banos_preciopromedio, "banos", "banos_precio_promedio")

In [18]:
train = asignar_caracteristica_precio_promedio(train, habitaciones_preciopromedio, "habitaciones", "habitaciones_precio_promedio")

In [19]:
train = asignar_caracteristica_precio_promedio(train, garages_preciopromedio, "garages", "garages_precio_promedio")

In [20]:
train_banos_preciopromedio = train.copy()
train_banos_preciopromedio = train_banos_preciopromedio.reset_index()
train_banos_preciopromedio = train_banos_preciopromedio[["id", "banos_precio_promedio"]]
train_banos_preciopromedio.head()

,id,banos_precio_promedio
0,254099,3.287627e+06
1,53461,3.962764e+06
2,247984,1.009003e+06
3,209067,6.749239e+05
4,185997,9.681813e+05


In [21]:
train_habitaciones_preciopromedio = train.copy()
train_habitaciones_preciopromedio = train_habitaciones_preciopromedio.reset_index()
train_habitaciones_preciopromedio = train_habitaciones_preciopromedio[["id", "habitaciones_precio_promedio"]]
train_habitaciones_preciopromedio.head()

,id,habitaciones_precio_promedio
0,254099,2.630239e+06
1,53461,5.066960e+06
2,247984,8.680177e+05
3,209067,8.180119e+05
4,185997,1.306128e+06


In [22]:
train_garages_preciopromedio = train.copy()
train_garages_preciopromedio = train_garages_preciopromedio.reset_index()
train_garages_preciopromedio = train_garages_preciopromedio[["id", "garages_precio_promedio"]]
train_garages_preciopromedio.head()

,id,garages_precio_promedio
0,254099,2.344480e+06
1,53461,4.060166e+06
2,247984,9.887201e+05
3,209067,9.262833e+05
4,185997,1.055876e+06


In [23]:
import common.features_precio_promedio as feature

In [24]:
train = pd.read_csv('sets_de_datos/train.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/test.csv', index_col = 0)

In [25]:
train = feature.asignar_precio_promedio_por_cantidad_de_banos_por_ciudad(train)

In [27]:
train

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,banos_precio_promedio
id,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,...,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0,3.287627e+06
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,...,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0,3.962764e+06
247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,...,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0,1.009003e+06
209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,...,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0,6.749239e+05
185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,...,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,9.681813e+05
126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,1.0,...,19.300612,-99.148475,2014-03-18 00:00:00,0.0,0.0,0.0,0.0,1.0,1100000.0,1.558578e+06
139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,2.0,...,17.143504,-96.803504,2016-02-23 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,1.828483e+06
5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,3.0,...,20.672612,-87.037968,2016-10-20 00:00:00,0.0,0.0,0.0,0.0,0.0,4200000.0,3.145881e+06
44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,1.0,...,NaN,NaN,2014-01-06 00:00:00,0.0,0.0,0.0,1.0,1.0,310000.0,5.730477e+05
